In [1]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from tensorflow.keras.utils import to_categorical


# Load and preprocess the dataset
def load_and_preprocess_data():
    # Load data
    X_train = np.loadtxt("../../Datasets/CAPP Dataset/SubjectDependent50PercentOverlap/X_train.txt")
    y_train = np.loadtxt("../../Datasets/CAPP Dataset/SubjectDependent50PercentOverlap/y_train.txt")
    X_test = np.loadtxt("../../Datasets/CAPP Dataset/SubjectDependent50PercentOverlap/X_test.txt")
    y_test = np.loadtxt("../../Datasets/CAPP Dataset/SubjectDependent50PercentOverlap/y_test.txt")

    # Reshape data (assuming 100 time steps and 9 features)
    X_train = X_train.reshape(X_train.shape[0], 100, 9)
    X_test = X_test.reshape(X_test.shape[0], 100, 9)

    # Normalize features
    scaler = StandardScaler()
    X_train_reshaped = X_train.reshape(-1, X_train.shape[-1])
    X_test_reshaped = X_test.reshape(-1, X_test.shape[-1])
    X_train_scaled = scaler.fit_transform(X_train_reshaped).reshape(X_train.shape)
    X_test_scaled = scaler.transform(X_test_reshaped).reshape(X_test.shape)

    # Find the actual number of unique classes
    all_labels = np.unique(np.concatenate((y_train, y_test)))
    num_classes = len(all_labels)
    print(f"Unique labels: {all_labels}")
    print(f"Number of classes: {num_classes}")

    # Create a mapping from original labels to consecutive integers
    label_map = {label: i for i, label in enumerate(all_labels)}

    # Apply the mapping to y_train and y_test
    y_train_mapped = np.array([label_map[label] for label in y_train])
    y_test_mapped = np.array([label_map[label] for label in y_test])

    # One-hot encode the mapped labels
    y_train_onehot = to_categorical(y_train_mapped, num_classes=num_classes)
    y_test_onehot = to_categorical(y_test_mapped, num_classes=num_classes)

    print(f"One-hot encoded label shape: {y_train_onehot.shape[1]}")

    return X_train_scaled, y_train_onehot, X_test_scaled, y_test_onehot, num_classes, label_map


# Create the TFLite Micro compatible model
def create_model(input_shape, num_classes):
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=input_shape),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Reshape((input_shape[0], input_shape[1], 1)),  # Reshape to 4D tensor for Conv2D
        tf.keras.layers.Conv2D(32, (1, 1), activation='relu'),  # 2D Convolutional layer
        tf.keras.layers.GlobalAveragePooling2D(),  # Global average pooling
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])
    return model


# Train the model
def train_model(model, X_train, y_train, X_val, y_val, epochs=50, batch_size=32):
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    history = model.fit(X_train, y_train,
                        validation_data=(X_val, y_val),
                        epochs=epochs,
                        batch_size=batch_size,
                        callbacks=[early_stopping],
                        verbose=1)
    return history


def evaluate_model(model, X_test, y_test):
    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
    print(f"Test accuracy: {accuracy:.4f}")

    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_test_classes = np.argmax(y_test, axis=1)

    # Convert back to original labels for the classification report
    inv_label_map = {v: k for k, v in label_map.items()}
    y_pred_original = np.array([inv_label_map[label] for label in y_pred_classes])
    y_test_original = np.array([inv_label_map[label] for label in y_test_classes])

    print(classification_report(y_test_original, y_pred_original, digits=5))


# Load and preprocess data
X_train, y_train, X_test, y_test, num_classes, label_map = load_and_preprocess_data()
print("num_classes: ", num_classes)

# Split training data into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Create and train the model
input_shape = X_train.shape[1:]
CNN_MLP = create_model(input_shape, num_classes)
history = train_model(CNN_MLP, X_train, y_train, X_val, y_val)

# Evaluate the model
evaluate_model(CNN_MLP, X_test, y_test)

2024-09-27 11:14:05.929649: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-27 11:14:05.941854: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-27 11:14:05.946072: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-27 11:14:05.955910: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-27 11:14:06.465699: W tensorflow/compiler/tf2

Unique labels: [ 0.  1.  2.  3.  6.  7.  8.  9. 10. 11. 12. 13. 14. 19. 20. 21. 22. 23.
 25. 26. 27.]
Number of classes: 21
One-hot encoded label shape: 21
num_classes:  21


I0000 00:00:1727414069.434858  181198 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1727414069.473343  181198 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1727414069.476822  181198 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1727414069.483690  181198 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

ValueError: The total size of the tensor must be unchanged. Received: input_shape=(100, 32), target_shape=(100, 9, 1)

In [2]:
# Convert the model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(CNN_MLP)
tflite_model = converter.convert()

# Save the TFLite model
with open("GeneratedTFLiteFilesAndOGModels/conv_mlp_model.tflite", "wb") as f:
    f.write(tflite_model)

print("TensorFlow Lite model has been saved as 'conv_mlp_model.tflite'")

# Optional: Save the Keras model
CNN_MLP.save("GeneratedTFLiteFilesAndOGModels/conv_mlp_model.h5")
print("Keras model has been saved as 'conv_mlp_model.h5'")

try:
    from everywhereml.code_generators.tensorflow import convert_model

    c_header = convert_model(CNN_MLP, X_test, y_test, model_name='conv_mlp_model')

    with open("GeneratedHeaderFiles/conv_mlp_model.h", "w") as file:
        file.write(c_header)

    print("C header file has been saved as GeneratedHeaderFiles/conv_mlp_model.h'")
except ImportError:
    print("everywhereml library not found. Skipping C header file generation.")

INFO:tensorflow:Assets written to: /tmp/tmpimry2hpj/assets


INFO:tensorflow:Assets written to: /tmp/tmpimry2hpj/assets


Saved artifact at '/tmp/tmpimry2hpj'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 100, 9), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 21), dtype=tf.float32, name=None)
Captures:
  131626278675808: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131626278679504: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131626274226416: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131626274231168: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131626274230288: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131626274233104: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131626274226944: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131626274234864: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131626274228880: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131626274236448: TensorSpec(shape=(), dtype=tf.resource, name=None)


W0000 00:00:1727413813.107343  176270 tf_tfl_flatbuffer_helpers.cc:392] Ignored output_format.
W0000 00:00:1727413813.107356  176270 tf_tfl_flatbuffer_helpers.cc:395] Ignored drop_control_dependency.
2024-09-27 11:10:13.107552: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpimry2hpj
2024-09-27 11:10:13.107852: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2024-09-27 11:10:13.107860: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmpimry2hpj
2024-09-27 11:10:13.110668: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2024-09-27 11:10:13.111237: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2024-09-27 11:10:13.130381: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /tmp/tmpimry2hpj
2024-09-27 11:10:13.135869: I tensorflow/cc/saved_model/loader.cc

TensorFlow Lite model has been saved as 'conv_mlp_model.tflite'
Keras model has been saved as 'conv_mlp_model.h5'
INFO:tensorflow:Assets written to: /tmp/tmpt7jkhus_/assets


INFO:tensorflow:Assets written to: /tmp/tmpt7jkhus_/assets


Saved artifact at '/tmp/tmpt7jkhus_'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 100, 9), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 21), dtype=tf.float32, name=None)
Captures:
  131626278675808: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131626278679504: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131626274226416: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131626274231168: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131626274230288: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131626274233104: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131626274226944: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131626274234864: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131626274228880: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131626274236448: TensorSpec(shape=(), dtype=tf.resource, name=None)


W0000 00:00:1727413813.495107  176270 tf_tfl_flatbuffer_helpers.cc:392] Ignored output_format.
W0000 00:00:1727413813.495118  176270 tf_tfl_flatbuffer_helpers.cc:395] Ignored drop_control_dependency.
2024-09-27 11:10:13.495229: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpt7jkhus_
2024-09-27 11:10:13.495529: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2024-09-27 11:10:13.495537: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmpt7jkhus_
2024-09-27 11:10:13.498407: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2024-09-27 11:10:13.517347: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /tmp/tmpt7jkhus_
2024-09-27 11:10:13.523066: I tensorflow/cc/saved_model/loader.cc:462] SavedModel load for tags { serve }; Status: success: OK. Took 27839 microseconds.


C header file has been saved as GeneratedHeaderFiles/conv_mlp_model.h'
